In [1]:
import sys
# sys.path.append('/content/drive/MyDrive/Colab Notebooks')

import time
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

# My modules
from optimization import define_optim
from model import MyModel
from data_load import load_dataset
from visualization import plot_loss_history, plot_accuracy_history, my_display_confusion_matrix
from metrics import Metrics
from testing import final_testing


# Configuration
config = {
    "batch_size": 32,
    "num_epochs": 20,
    "learning_rate": 3e-4,
    "scheduler_step": 1,
    "scheduler_gamma": 0.98,
    "weight_decay": 8e-4,
    "dropout_p": 0,
    "num_classes": None,
    "name_classes": None,
    "dataset_path": 'simpsons_dataset',
    "device": 'cpu'
}

if __name__ == "__main__":
    # Dataset folding
    print("Data loading...")
    
    # Creating a loader
    start = time.time()
    loader, train_set, test_set = load_dataset(config)
    config["device"] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Model loading
    writer = SummaryWriter("runs/without_dropout_0")
    writer.add_text("config", str(config), 0)
    model = MyModel(config, writer)
    
    # Define the loss function, optimizer and learning rate scheduler
    criterion, optimizer, scheduler = define_optim(model.model, config)
    metrics = Metrics(config)
    print('{:.2f}s'.format(time.time() - start))
    print('Processing...')
    
    # Main loop
    for epoch in range(config["num_epochs"]):
        start = time.time()
        model.train_step(epoch, loader, optimizer, criterion, scheduler)
        model.test_step(epoch, loader, criterion, metrics)
        
        # Macro average metrics
        print('Epoch: {}\nPrecision: {:.5f}\nRecall: {:.5f}\nAccuracy: {:.5f}\nTime: {:.2f}s\n'.
              format(epoch + 1,
                     metrics.precision().mean(),
                     metrics.recall().mean(),
                     metrics.accuracy().mean(),
                     time.time() - start))
        writer.add_scalar("recall", metrics.recall().mean(), epoch)
        writer.add_scalar("precision", metrics.precision().mean(), epoch)
        if epoch != config["num_epochs"] - 1:
            metrics.reset_metrics()
    model.save("runs/mymodel.pth")



Data loading...
3.00s
Processing...
Epoch: 1
Precision: 0.80034
Recall: 0.70825
Accuracy: 0.99762
Time: 58.75s

Epoch: 2
Precision: 0.84288
Recall: 0.77133
Accuracy: 0.99765
Time: 56.57s

Epoch: 3
Precision: 0.93483
Recall: 0.88273
Accuracy: 0.99832
Time: 56.11s

Epoch: 4
Precision: 0.91243
Recall: 0.84077
Accuracy: 0.99821
Time: 56.79s

Epoch: 5
Precision: 0.96505
Recall: 0.84188
Accuracy: 0.99815
Time: 56.28s

Epoch: 6
Precision: 0.93896
Recall: 0.87195
Accuracy: 0.99826
Time: 57.04s

Epoch: 7
Precision: 0.95261
Recall: 0.90759
Accuracy: 0.99859
Time: 55.95s

Epoch: 8
Precision: 0.94863
Recall: 0.91545
Accuracy: 0.99843
Time: 56.76s

Epoch: 9
Precision: 0.93499
Recall: 0.89527
Accuracy: 0.99858
Time: 56.18s

Epoch: 10
Precision: 0.93239
Recall: 0.89143
Accuracy: 0.99814
Time: 56.92s

Epoch: 11
Precision: 0.93223
Recall: 0.91552
Accuracy: 0.99864
Time: 57.24s

Epoch: 12
Precision: 0.94261
Recall: 0.89341
Accuracy: 0.99835
Time: 56.42s

Epoch: 13
Precision: 0.94531
Recall: 0.91955
Accu

In [2]:
'''test_model = model 
plot_loss_history(test_model.get_loss_history())
plot_accuracy_history(test_model.get_accuracy_history())
my_display_confusion_matrix(metrics.get_confusion_matrix(), dataset.classes)
final_testing("runs/mymodel.pth", dataset.classes, loader, device)
plt.show()'''

'test_model = model \nplot_loss_history(test_model.get_loss_history())\nplot_accuracy_history(test_model.get_accuracy_history())\nmy_display_confusion_matrix(metrics.get_confusion_matrix(), dataset.classes)\nfinal_testing("runs/mymodel.pth", dataset.classes, loader, device)\nplt.show()'

In [3]:
print(metrics.accuracy())

[0.99761735 1.         0.99904694 0.99857041 0.99594949 0.99976173
 0.99809388 0.99618775 0.99952347 0.99046938 1.         0.99857041
 0.99952347 0.99976173 0.99904694 0.99428163 0.99904694 0.99833214
 0.99880867 0.9992852  0.99737908 0.99880867 0.99737908 0.99904694
 0.99880867 0.99904694 0.9992852  0.99642602 0.99690255 0.99737908
 1.         0.99833214 0.99761735 0.99904694 0.9992852  1.
 0.99880867 0.99833214 1.         0.9992852  1.         0.99880867]


In [4]:
print(metrics.recall())

[0.96875    1.         0.98611111 0.75       0.996      0.91666667
 0.98190045 0.9408867  0.83333333 1.         1.         0.97777778
 1.         0.85714286 0.9        0.96642686 0.99029126 0.98319328
 1.         0.57142857 0.97785978 0.79166667 0.96934866 1.
 0.92592593 0.99557522 0.66666667 0.96091205 0.96478873 0.93243243
 1.         0.6        0.97590361 0.75       0.93333333 1.
 0.77777778 0.98245614 1.         0.75       1.         0.89583333]


In [5]:
print(metrics.precision())

[0.97894737 1.         0.98611111 0.9375     0.93962264 1.
 0.98190045 0.97948718 1.         0.69230769 1.         0.95652174
 0.71428571 1.         0.9        0.97578692 0.97142857 0.98734177
 0.92424242 1.         0.98148148 1.         0.98828125 0.80952381
 0.98039216 0.98684211 0.8        0.98993289 0.98916968 0.92
 1.         0.66666667 0.98380567 1.         0.875      1.
 0.93333333 0.97674419 1.         1.         1.         1.        ]
